# Run Colab on a Coral Dev Board

This tutorial shows you how to run Jupyter notebooks on your Coral Dev Board, and then edit and execute them from your connected computer using the [Google Colab interface](https://research.google.com/colaboratory/faq.html).

By the end of this tutorial, you'll connect this notebook to your Dev Board and run TensorFlow Lite code directly from this page on the board.


## Requirements

+ You need a Coral Dev Board and it should be [set up with the latest software](https://coral.ai/docs/dev-board/get-started/).

  If you run `cat /etc/mendel_version` on the Dev Board, it should print 4.0 or higher.

+ The computer you're using to read this tutorial must have [MDT installed](https://coral.ai/docs/dev-board/mdt/).

## Install Jupyter

First you need to install Jupyter on your Dev Board as follows:

1.  Open a shell on your Dev Board:

    ```bash
    mdt shell
    ```

2.  Install the Python 3 development tools on the board:

    ```bash
    sudo apt-get update

    sudo apt-get install python3-dev
    ```

3.  Install Jupyter:

    ```bash
    pip3 install jupyter
    ```

4.  Reload the `.profile` file to access Jupyter's location (at `~/.local/bin`):

    ```
    source $HOME/.profile
    ```

5.  Verify it works by running `jupyter --version`.

    If this prints `command not found`, then your Mendel version might be out of date or the Jupyter installation failed.








## Start Jupyter on the Dev Board

Before starting Jupyter on the Dev Board, you need to install `jupyter_http_over_ws`, which enables your host computer to communicate with Jupyter via WebSocket:

```bash
pip3 install jupyter_http_over_ws

jupyter serverextension enable --py jupyter_http_over_ws
```

Once that's done, start Jupyter as follows:

```bash
jupyter notebook \
  --NotebookApp.allow_origin='https://colab.research.google.com' \
  --port=8888 \
  --NotebookApp.port_retries=0
```

This outputs several messages, including a URL such as `http://localhost:8888/?token=...`. You'll
need to come back and copy this token later.

Be sure you leave this terminal open because it's now running the Jupyter notebook.

## Start SSH port forwarding

Although Jupyter is now running, it's only accessible from the Dev Board. To access it from the host computer, you need to forward your localhost traffic to the Dev Board with an SSH tunnel. The effect is to make Google Colab think Jupyter notebook is running on your host computer even though it's actually running on the Dev Board.

Open a new terminal on your host computer (where you run `mdt shell`) and run this command:

```bash
ssh -N -L 8888:localhost:8888 mendel@192.168.100.2 -i ~/.config/mdt/keys/mdt.key
```

If asked, `Are you sure you want to continue connecting?`, type `yes`.

If you see a warning that says `REMOTE HOST IDENTIFICATION HAS CHANGED`, then you need to open the `known_hosts` file and remove the line specifying the key for `192.168.100.2`. Then try again.

Now leave this window active because this command is what keeps the SSH tunnel open between the Dev Board and the host computer.



## Connect this Colab to the local runtime

1.  Open http://localhost:8888 in a browser on the host computer.
    
2.  You should see a Jupyter notebook page asking for "Password or token" at the top. Paste the token printed by the `jupyter notebook` command above (just the part following `?token=`), and click **Log in**.

2.  Now, on *this tutorial page's* Colab interface, click the drop-down arrow next to **Connect** (near the top-right corner of the web page) and click **Connect to local runtime**.

4.  Use `8888` as the port and click **Connect**.

    The button should now say "Connected (Local)".

This notebook (the page you're reading) is now connected to your Dev Board and you can run code from this page on the board!


## Run notebook code on the Dev Board


You can execute the code blocks below one at a time or run them all by selecting **Runtime > Run all** in the toolbar.

First you need to download pre-compiled models and data from the Edge TPU API examples:

In [1]:
! sudo apt-get update

! sudo apt-get install edgetpu-examples

Get:1 file:/var/cache/mdt/packages ./ InRelease
Ign:1 file:/var/cache/mdt/packages ./ InRelease
Get:2 file:/var/cache/mdt/packages ./ Release
Ign:2 file:/var/cache/mdt/packages ./ Release
Get:3 file:/var/cache/mdt/packages ./ Packages
Ign:3 file:/var/cache/mdt/packages ./ Packages
Get:4 file:/var/cache/mdt/packages ./ Translation-en
Ign:4 file:/var/cache/mdt/packages ./ Translation-en
Get:5 file:/var/cache/mdt/packages ./ Translation-en_US
Ign:5 file:/var/cache/mdt/packages ./ Translation-en_US
Get:3 file:/var/cache/mdt/packages ./ Packages
Ign:3 file:/var/cache/mdt/packages ./ Packages
Get:4 file:/var/cache/mdt/packages ./ Translation-en
Ign:4 file:/var/cache/mdt/packages ./ Translation-en
Get:5 file:/var/cache/mdt/packages ./ Translation-en_US
Ign:5 file:/var/cache/mdt/packages ./ Translation-en_US
Get:3 file:/var/cache/mdt/packages ./ Packages
Ign:3 file:/var/cache/mdt/packages ./ Packages
Get:4 file:/var/cache/mdt/packages ./ Translation-en
Ign:4 file:/var/cache/mdt/packages ./ Tra

Then you can run an inference with this code:

In [2]:
from edgetpu.classification.engine import ClassificationEngine
from edgetpu.utils import dataset_utils
from PIL import Image

# Prepare labels.
labels = dataset_utils.read_label_file('/usr/share/edgetpu/examples/models/inat_bird_labels.txt')
# Initialize engine.
engine = ClassificationEngine('/usr/share/edgetpu/examples/models/mobilenet_v2_1.0_224_inat_bird_quant_edgetpu.tflite')
# Run inference.
img = Image.open('/usr/share/edgetpu/examples/images/parrot.jpg')
for result in engine.classify_with_image(img, top_k=3):
 print('---------------------------')
 print(labels[result[0]])
 print('Score : ', result[1])

---------------------------
Ara macao (Scarlet Macaw)
Score :  0.61328125
---------------------------
Platycercus elegans (Crimson Rosella)
Score :  0.15234375


You should see classification results for "Ara macao (Scarlet Macaw)." 

You can now create your own Colab files and connect them to the same local runtime.